# Activ4b dataset decoding
The datasets of Activ4b (phs002710) were provided as CSV files that contained encoded data. For the BioData Catalyst PIC-SURE ETL pipeline, these files need to be in the decoded format. The purpose of this notebook is to use the Data Dictionary, provided in Excel format, to decode the CSV files and save them to the S3 bucket.

## Do imports and set user-defined functions

In [ ]:
# Do imports
import pandas as pd
import glob

In [ ]:
# Set directories and get file information
data_dict_path = "/home/ec2-user/SageMaker/pic-sure-metadata-curation/activ4b/input/DataDict.xlsx"
file_dir = "/home/ec2-user/studies/ALL-avillach-73-bdcatalyst-etl/activ4b/development_raw_data/*"
out_dir = "/home/ec2-user/studies/ALL-avillach-73-bdcatalyst-etl/activ4b/development_decoded_data/"
file_prefixes = [i.split('/')[-1].strip(".csv") for i in glob.glob(file_dir)]

In [ ]:
file_prefixes